In [ ]:
!pip install formula

     |████████████████████████████████| 24.3 MB 17.1 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for formula: filename=formula-2.0.1-cp37-cp37m-linux_x86_64.whl size=1104718 sha256=493f62805544ac7316f0f1f5ea58d6a00c74594b1918e6b5350b57d5625d3d4a
  Stored in directory: /root/.cache/pip/wheels/dd/ed/a9/3962025d76b8dbf796a5d02985ffa66a3848cf2add259869f7
Successfully built formula


In [ ]:
class Formula:
	def __invert__(self):
		return Not(self)
	def __and__(self, other):
		return And(self, other)
	def __or__(self, other):
		return Or(self, other)
	def __rshift__(self, other):
		return Implies(self, other)
	def __lshift__(self, other):
		return Iff(self, other)
	def __eq__(self, other):
		return self.__class__ == other.__class__ and self.eq(other)
	def v(self, v):
		raise NotImplementedError("Plain formula can not be valuated")
	def _t(self, left, right):
		while True:
			found = True
			for item in left:
				if item in right:
					return None
				if not isinstance(item, Atom):
					left.remove(item)
					tup = item._tleft(left, right)
					left, right = tup[0]
					if len(tup) > 1:
						v = self._t(*tup[1])
						if v is not None:
							return v
					found = False
					break
			for item in right:
				if item in left:
					return None
				if not isinstance(item, Atom):
					right.remove(item)
					tup = item._tright(left, right)
					left, right = tup[0]
					if len(tup) > 1:
						v = self._t(*tup[1])
						if v is not None:
							return v
					found = False
					break
			if found:
				return set(left)
	def t(self):
		return self._t([], [self])

class BinOp(Formula):
	def __init__(self, lchild, rchild):
		self.lchild = lchild
		self.rchild = rchild
	def __str__(self):
		return '(' + str(self.lchild) + ' ' + self.op+ ' ' + str(self.rchild) + ')'
	def eq(self, other):
		return self.lchild == other.lchild and self.rchild == other.rchild

class And(BinOp):
	op = '∧'
	def v(self, v):
		return self.lchild.v(v) and self.rchild.v(v)
	def _tleft(self, left, right):
		return (left + [self.lchild, self.rchild], right),
	def _tright(self, left, right):
		return (left, right + [self.lchild]), (left, right + [self.rchild])

class Or(BinOp):
	op = '∨'
	def v(self, v):
		return self.lchild.v(v) or self.rchild.v(v)
	def _tleft(self, left, right):
		return (left + [self.lchild], right), (left + [self.rchild], right)
	def _tright(self, left, right):
		return (left, right + [self.lchild, self.rchild]),

class Implies(BinOp):
	op = '→'
	def v(self, v):
		return not self.lchild.v(v) or self.rchild.v(v)
	def _tleft(self, left, right):
		return (left + [self.rchild], right), (left, right + [self.lchild])
	def _tright(self, left, right):
		return (left + [self.lchild], right + [self.rchild]),

class Iff(BinOp):
	op = '↔'
	def v(self, v):
		return self.lchild.v(v) is self.rchild.v(v)
	def _tleft(self, left, right):
		return (left + [self.lchild, self.rchild], right), (left, right + [self.lchild, self.rchild])
	def _tright(self, left, right):
		return (left + [self.lchild], right + [self.rchild]), (left + [self.rchild], right + [self.lchild])

class Not(Formula):
	def __init__(self, child):
		self.child = child
	def v(self, v):
		return not self.child.v(v)
	def __str__(self):
		return '¬' + str(self.child)
	def eq(self, other):
		return self.child == other.child
	def _tleft(self, left, right):
		return (left, right + [self.child]),
	def _tright(self, left, right):
		return (left + [self.child], right),

class Atom(Formula):
	def __init__(self, name):
		self.name = name
	def __hash__(self):
		return hash(self.name)
	def v(self, v):
		return self in v
	def __str__(self):
		return str(self.name)
	__repr__ = __str__
	def eq(self, other):
		return self.name == other.name

a = Atom('a')
b = Atom('b')
c = Atom('c')

def dop(f, e):
	print("Formula: ", f)
	print("Valuation for", e, ": ", f.v(e))
	print("Counterexample: ", f.t())

dop(a | b, {a})
dop(a >> b, {a})
dop(a << b, {a})
dop(a & b, {a,b})
dop(a & b >> (c >> a), {b,c})
dop(a & b | b & c, {b,c})
dop(~a & ~~~b, {})
dop(a >> (b >> c), {a, b})
dop(a >> (b >> c), {a, b, c})
dop(a >> b >> c, {a, c})
dop(((c | ~b) >> (b | c)) >> (b | c), {a, c})
dop(a | ~a, {})
dop(a >> a, {a})
dop(a << a, {})
dop((a >> b) | (b >> a), {})
dop((~a | b) | (~b | a), {})
dop((~a | a) | (~b | b), {})

Formula:  (a ∨ b)
Valuation for {a} :  True
Counterexample:  set()
Formula:  (a → b)
Valuation for {a} :  False
Counterexample:  {a}
Formula:  (a ↔ b)
Valuation for {a} :  False
Counterexample:  {b}
Formula:  (a ∧ b)
Valuation for {b, a} :  True
Counterexample:  set()
Formula:  (a ∧ (b → (c → a)))
Valuation for {b, c} :  False
Counterexample:  {b, c}
Formula:  ((a ∧ b) ∨ (b ∧ c))
Valuation for {b, c} :  True
Counterexample:  set()
Formula:  (¬a ∧ ¬¬¬b)
Valuation for {} :  True
Counterexample:  {b}
Formula:  (a → (b → c))
Valuation for {b, a} :  False
Counterexample:  {b, a}
Formula:  (a → (b → c))
Valuation for {b, a, c} :  True
Counterexample:  {b, a}
Formula:  ((a → b) → c)
Valuation for {a, c} :  True
Counterexample:  set()
Formula:  (((c ∨ ¬b) → (b ∨ c)) → (b ∨ c))
Valuation for {a, c} :  True
Counterexample:  None
Formula:  (a ∨ ¬a)
Valuation for {} :  True
Counterexample:  None
Formula:  (a → a)
Valuation for {a} :  True
Counterexample:  None
Formula:  (a ↔ a)
Valuation for {} : 

In [ ]:
!pip install Atom

     |████████████████████████████████| 1.6 MB 7.4 MB/s 
